In [1]:
import pandas
df = pandas.read_csv('salary-train.csv')

In [13]:
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [14]:
df['FullDescription'] = df['FullDescription'].str.lower()

In [17]:
df['FullDescription'] = df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [18]:
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [20]:
import sklearn
from sklearn import feature_extraction
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(min_df=5)

In [21]:
X = vectorizer.fit_transform(df['FullDescription'])

In [22]:
df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)

In [23]:
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [24]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))
#X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [29]:
type(X)

scipy.sparse.csr.csr_matrix

In [30]:
import scipy.sparse

In [33]:
X.shape, X_train_categ.shape

((60000, 91118), (60000, 1766))

In [34]:
X_train_final = scipy.sparse.hstack([X, X_train_categ])

In [35]:
X_train_final.shape

(60000, 92884)

In [37]:
Y_train = df['SalaryNormalized']

In [36]:
import sklearn.linear_model
ridge = sklearn.linear_model.Ridge(alpha=1, random_state=241)

In [38]:
ridge.fit(X_train_final, Y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [48]:
test_df = pandas.read_csv('test.csv')
test_df['FullDescription'] = test_df['FullDescription'].str.lower()
test_df['FullDescription'] = test_df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
X_test = vectorizer.transform(test_df['FullDescription'])
X_test_categ = enc.transform(test_df[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_final = scipy.sparse.hstack([X_test, X_test_categ])

In [49]:
X_test_final

<2x92884 sparse matrix of type '<type 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [51]:
ridge.predict(X_test_final)

array([ 56941.81748253,  37495.62950937])